# Functions

## Item 14: Prefer Exception to Returning None

- Functions that return None to indicate special meaning are error prone because None and other values (e.g., zero, the empty string) all evaluate to False in conditional expressions.

- Raise exceptions to indicate special situations instead of returning None. Expect the calling code to handle exceptions properly when they’re documented.

In [2]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e
        
x, y = 5, 0
try:
    result = divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.1f' % result)

Invalid inputs


## Item 15: Know How Closures Interact with Variable Scope
- Closure functions can refer to variables from any of the scopes in which they were defined.

- By default, closures can’t affect enclosing scopes by assigning variables.

- In Python 3, use the nonlocal statement to indicate when a closure can modify a variable in its enclosing scopes.

- In Python 2, use a mutable value (like a single-item list) to work around the lack of the nonlocal statement.

- Avoid using nonlocal statements for anything beyond simple functions.

In [5]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}

# use for Python 3
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

found = sort_priority3(numbers, group)
print('Found:', found)
print(numbers)

Found: True
[2, 3, 5, 7, 1, 4, 6, 8]


In [6]:
# Use for Python 2
def sort_priority(numbers, group):
    found = [False]
    def helper(x):
        if x in group:
            found[0] = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found[0]

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = set([2, 3, 5, 7])
found = sort_priority(numbers, group)
print('Found:', found)
print(numbers)

Found: True
[2, 3, 5, 7, 1, 4, 6, 8]


## Item 16: Consider Generators instead of Returning Lists
- Using generators can be clearer than the alternative of returning lists of accumulated results.

- The iterator returned by a generator produces the set of values passed to yield expressions within the generator function’s body.

- Generators can produce a sequence of outputs for arbitrarily large inputs because their working memory doesn’t include all inputs and outputs.

In [8]:
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

address = 'Four score and seven years ago our fathers brought forth on this continent a new nation, conceived in liberty, and dedicated to the proposition that all men are created equal.'

result = list(index_words_iter(address))
print(result[:3])

[0, 5, 11]


## Item 17: Be Defensive When Iterating Over Arguments

- Beware of functions that iterate over input arguments multiple times. If these arguments are iterators, you may see strange behavior and missing values.

- Python’s iterator protocol defines how containers and iterators interact with the iter and next built-in functions, for loops, and related expressions.

- You can easily define your own iterable container type by implementing the __iter__ method as a generator.

- You can detect that a value is an iterator (instead of a container) if calling iter on it twice produces the same result, which can then be progressed with the next built- in function.

In [11]:
def normalize_defensive(numbers):
    if iter(numbers) is iter(numbers):  # An iterator -- bad!
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

visits = [15, 35, 80]
normalize_defensive(visits)  # No error

[11.538461538461538, 26.923076923076923, 61.53846153846154]

In [14]:
try:
    it = iter(visits)
    normalize_defensive(it)
except:
    print("Have an error")
else:
    assert False

Have an error


## Item 18: Reduce Visual Noise with Variable Positional Arguments

- Functions can accept a variable number of positional arguments by using *args in the def statement.

- You can use the items from a sequence as the positional arguments for a function with the * operator.

- Using the * operator with a generator may cause your program to run out of memory and crash.

- Adding new positional parameters to functions that accept *args can introduce hard-to-find bugs.

In [15]:
def log(message, *values):  # The only difference
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print('%s: %s' % (message, values_str))

log('My numbers are', 1, 2)
log('Hi there')  # Much better

My numbers are: 1, 2
Hi there


## Item 19: Provide Optional Behavior with keyword Argument
- Function arguments can be specified by position or by keyword.

- Keywords make it clear what the purpose of each argument is when it would be confusing with only positional arguments.

- Keyword arguments with default values make it easy to add new behaviors to a function, especially when the function has existing callers.

- Optional keyword arguments should always be passed by keyword instead of by position.

In [16]:
def remainder(number, divisor):
    return number % divisor

remainder(20, 7)
remainder(20, divisor=7) # position arugments always before keyword arguments
remainder(number=20, divisor=7)
remainder(divisor=7, number=20)

6

## Item 20: Use None and Docstrings to Specify dynamic default Arguments

- Default arguments are only evaluated once: during function definition at module load time. This can cause odd behaviors for dynamic values (like {} or []).

- Use None as the default value for keyword arguments that have a dynamic value. Document the actual default behavior in the function’s docstring.

In [20]:
from datetime import datetime
from time import sleep

def log(message, when=None):
    """Log a message with a timestamp.
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    when = datetime.now() if when is None else when
    print('%s: %s' % (when, message))


# Example 3
log('Hi there!')
sleep(0.1)
log('Hi again!')


2017-03-22 10:18:08.232441: Hi there!
2017-03-22 10:18:08.334322: Hi again!


In [22]:
import json

def decode(data, default=None):
    """Load JSON data from a string.
    Args:
        data: JSON data to decode.
        default: Value to return if decoding fails.
            Defaults to an empty dictionary.
    """
    if default is None:
        default = {}
    try:
        return json.loads(data)
    except ValueError:
        return default
    
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)

Foo: {'stuff': 5}
Bar: {'meep': 1}


## Item 21: Enforce Clarity with Keyword Only Arguments

- Keyword arguments make the intention of a function call more clear.

- Use keyword-only arguments to force callers to supply keyword arguments for potentially confusing functions, especially those that accept multiple Boolean flags.

- Python 3 supports explicit syntax for keyword-only arguments in functions.

In [23]:
def safe_division_c(number, divisor, *,
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

safe_division_c(1.0, 0, ignore_zero_division=True)  # No exception

inf

In [25]:
safe_division_c(1.0, 0, False, True)  # No exception

TypeError: safe_division_c() takes 2 positional arguments but 4 were given